In [34]:
import import_ipynb
import openai
from dotenv import load_dotenv
import requests
import re
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta, timezone
import locale
import unicodedata

In [35]:
load_dotenv()
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [36]:
class Infantil:
    
    def __init__(self, date, date_str):
        self.date = date
        self.date_str = date_str
        self.theme = self.get_theme()
        # self.news1_header = self.get_news1_header()
        # self.news1_text = self.get_news1_text()
        # self.news1_imageUrl = self.get_news1_image()
        self.news2_text = self.get_curiosity_text()
        # self.news2_imageUrl = self.get_curiosity_image(self.news2_text)
        self.news3_text = self.get_curiosity_text()
        # self.news3_imageUrl = self.get_curiosity_image(self.news3_text)
        

    def get_theme(self):
        url = 'https://www.ponteiro.com.br/todas_datas.php'
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_dates = soup.find_all('tr')
        
        calendar = []
        edition_str = self.date.strftime('%d/%m')
        for date in all_dates:
            day = date.find('td', width=" 40").get_text().strip() if date.find('td', width=" 40") != None else None
            val = date.find('td', width="490").get_text().strip() if date.find('td', width="490") != None else None
            if day == edition_str and val != None:
                calendar.append({"dia": day, "comemora": val})
        
        question = "Escolha o tema da seção infantil de hoje:"
        print("Opções de tema para a seção infantil do dia:")
        print(f'#0 - Personalizar tema')
        counter = 1
        for c in calendar:
            print(f'#{counter} - {c["comemora"]}')
            counter += 1
            
        print(question)
        
        selected_date = int(input(question))
        if selected_date == 0:
            theme = input(f'Digite o tema desejado para {self.date_str}:\n')
        else:
            theme = calendar[selected_date-1]["comemora"]

        # Continuar o processamento usando a entrada do usuário
        print(f"\nO tema escolhido foi: {theme}")

        return theme



    def get_news1_header(self):
        parts = self.date_str.split(' ')
        header = f'{parts[1]} de {parts[3]}: {self.theme}'
        return header

    
    def get_news1_text(self):
        content = f'Escreva um texto voltado para crianças recém alfabetizadas com o tema {self.theme}. ' 
        content += f'Esse texto estará na seção infantil do dia {self.date.strftime('%d/%m')} ao lado de uma imagem em preto em branco (para ser colorida) '
        content += f'sobre o tema. O texto deve conter entre 900 e 950 caracteres. Escreva estritamente o corpo do texto.'
        
        client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        message = [{
            'role': 'user',
            'content': content
        }]
        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=0, 
        )
        news_text = response.choices[0].message.content
        print(f'Infantil - Notícia 1: {header}')
        return news_text
    
    
    
    def get_news1_image(self):
        prompt = f'Crie uma imagem em preto e branco (no estilo line art) com o objetivo de ser colorida por crianças entre 2 e 6 anos. A imagem deve estar relacionada ao tema ' 
        prompt += f'{self.theme} e será divulgada na seção infantil de um jornal.'
        
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        image_url = response.data[0].url
        return image_url
        
        
        
    def get_curiosity_text(self):
        theme = input(f'Digite o tema desejado para a curiosidade:\n')
        content = f'Aponte-me uma curiosidade para inserir na seção infantil de um jornal. Forneça, como resposta, estritamente a curiosidade da maneira como ela deve ser escrita em um texto contento entre 800 e 850 caracteres. O tema é {theme}.'
        client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        message = [{
            'role': 'user',
            'content': content
        }]
        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=1, 
        )
        news_text = response.choices[0].message.content
        print(f'Infantil - Curiosidade: {theme}')
        return news_text
    
    
    
    def get_curiosity_image(self, curiosity):
        prompt = f'Crie uma imagem infantil para ser colocara junto ao texto a seguir na seção infantil de um jornal de grande circulação.\n\n' 
        prompt += f'{curiosity}'
        
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        image_url = response.data[0].url
        return image_url

        
